<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Set-parameters" data-toc-modified-id="Set-parameters-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Set parameters</a></span></li><li><span><a href="#Read-data-from-CDC" data-toc-modified-id="Read-data-from-CDC-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Read data from CDC</a></span></li><li><span><a href="#Create-a-2nd-df-to-manipulate" data-toc-modified-id="Create-a-2nd-df-to-manipulate-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Create a 2nd df to manipulate</a></span><ul class="toc-item"><li><span><a href="#Create-a-subset-based-on-date" data-toc-modified-id="Create-a-subset-based-on-date-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Create a subset based on date</a></span></li></ul></li><li><span><a href="#Create-a-df-with-just-data-from-Delaware" data-toc-modified-id="Create-a-df-with-just-data-from-Delaware-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create a df with just data from Delaware</a></span></li><li><span><a href="#Create-a-df-that-contains-data-summarized-by-year-and-month" data-toc-modified-id="Create-a-df-that-contains-data-summarized-by-year-and-month-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Create a df that contains data summarized by year and month</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#What-needs-to-change-if-we-want-to-the-same-for-v2DE-but-we-want-to-exclude-the-mean-and-include-dist_first,-dist_last?" data-toc-modified-id="What-needs-to-change-if-we-want-to-the-same-for-v2DE-but-we-want-to-exclude-the-mean-and-include-dist_first,-dist_last?-5.0.1"><span class="toc-item-num">5.0.1&nbsp;&nbsp;</span>What needs to change if we want to the same for v2DE but we want to exclude the mean and include dist_first, dist_last?</a></span></li></ul></li></ul></li><li><span><a href="#Write-out-each-df-as-a-csv-file" data-toc-modified-id="Write-out-each-df-as-a-csv-file-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Write out each df as a csv file</a></span></li></ul></div>

# Set parameters

In [1]:
# Step 1 - put keys into parameters
# Step 2 - put keys into .env file

#MyAppToken = ''
#Myusername = ''
#Mypassword = ''
cdc_api = 'unsk-b7fc'
row_limit = 150000
start_date = '2021-04-27'

keep_columns = ['date','mmwr_week','location','distributed','administered','distributed_janssen',
                'distributed_moderna','distributed_pfizer', 'additional_doses','administered_12plus',
                'administered_18plus','administered_65plus','series_complete_yes','series_complete_12plus',
                'series_complete_18plus','series_complete_65plus']

drop_columns = {'distributed_janssen',
                'distributed_moderna', 
                'distributed_pfizer'}

bin_values = [0,100000,1000000,100000000]
bin_labels = ['low','medium','high']

state = 'DE'


# Read data from CDC

In [2]:
import pandas as pd
import numpy as np
import requests
from sodapy import Socrata

########## NEW #################
from dotenv import load_dotenv
import os

load_dotenv()

MyAppToken = os.getenv('MyAppToken')
Myusername = os.getenv('Myusername')
Mypassword = os.getenv('Mypassword')

client = Socrata('data.cdc.gov',
                  MyAppToken,
                  username= Myusername,
                  password= Mypassword)

#results = client.get("8xkx-amqh", limit = 100000)
results = client.get(cdc_api, limit = row_limit)
vaccines = pd.DataFrame(results)

# This is where the documentation is:
# https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-Jurisdi/unsk-b7fc

In [3]:
#print(vaccines.columns)
print(vaccines.shape)

(21656, 69)


# Create a 2nd df to manipulate

## Create a subset based on date

In [ ]:
v2 = vaccines[vaccines.date >= start_date]
v2.shape

# At this point, date is an object not a date

In [ ]:
# Create a new dataframe with just the required columns
# new=  old[['A', 'C', 'D']]
v2 = vaccines[keep_columns]

In [ ]:
v2.head()

In [ ]:
v2.info()

In [ ]:
# Drop columns
v2 = v2.drop(columns = drop_columns)

In [ ]:
# Which columns have null values?
v2.isnull().sum()

In [ ]:
# Fill the Null values with zero

v2['additional_doses'] = v2['additional_doses'].fillna(0)
# Alternate code v2['additional_doses'].fillna(0, inplace = True)

v2.isnull().sum()

In [ ]:
#. Change the datatypes

v2['date'] = v2['date'].astype('datetime64[ns]')

#v2['distributed'] = pd.to_numeric(v2['distributed']).astype(int)
#v2['administered'] = pd.to_numeric(v2['administered']).astype(int)
#v2['additional_doses'] = pd.to_numeric(v2['additional_doses']).astype(int)
#v2['administered_12plus'] = pd.to_numeric(v2['administered_12plus']).astype(int)
#v2['administered_18plus'] = pd.to_numeric(v2['administered_18plus']).astype(int)
#v2['administered_65plus'] = pd.to_numeric(v2['administered_65plus']).astype(int)
#v2['series_complete_yes'] = pd.to_numeric(v2['series_complete_yes']).astype(int)
#v2['series_complete_12plus'] = pd.to_numeric(v2['series_complete_12plus']).astype(int)
#v2['series_complete_18plus'] = pd.to_numeric(v2['series_complete_18plus']).astype(int)
#v2['series_complete_65plus'] = pd.to_numeric(v2['series_complete_65plus']).astype(int)


In [ ]:
columns = list(v2.columns)

#change the first and last column names below as required
start_column = columns.index('distributed')
end_column   = columns.index('series_complete_65plus')

for index, col in enumerate(columns):
    if (start_column <= index) & (index <= end_column):
        v2[col] = v2[col].astype(int)


In [ ]:
v2.info()

In [ ]:
#Create year, month, day columns from date

v2['year']= v2['date'].dt.year
v2['month']= v2['date'].dt.month
v2['day']= v2['date'].dt.day
v2.head()

In [ ]:
# Create a column for season



In [ ]:
# Create a column to classify the distribution volume


v2['volume'] = pd.cut(v2['distributed'], bins=bin_values,labels=bin_labels,right = False)
v2.head()

# Create a df with just data from Delaware

In [ ]:
v2DE = v2[v2.location == state]

In [ ]:
v2DE = v2DE.sort_values('date')

In [ ]:
v2DE.shape

In [ ]:
v2DE.head()

In [ ]:
# How many vaccines were distributed at the beginning of the month?
v2DE['dist_first'] = v2DE.sort_values(by=['day']).groupby(['year', 'month'])['distributed'].transform('first')
v2DE['dist_last']  = v2DE.sort_values(by=['day']).groupby(['year', 'month'])['distributed'].transform('last')

In [ ]:
v2DE.head()

In [ ]:
v2DE[['date', 'mmwr_week','distributed','year','month','day','dist_first', 'dist_last']]

# Create a df that contains data summarized by year and month

In [ ]:
# Only aggregate distributed and administered

v2_agg = v2.groupby(['year','month']).agg({'distributed':['sum','mean'],'administered':['sum','mean']}).reset_index()
v2_agg.head()

# df_new = df.groupby(['col1', 'col2'])["col3", "col4"].sum()

### What needs to change if we want to the same for v2DE but we want to exclude the mean and include dist_first, dist_last?

In [ ]:
v2DE2 = v2DE.groupby(['year','month']).agg(distributed = ('distributed','sum'),
                                            administered = ('administered','sum'),
                                            dist_first = ('dist_first','min'),
                                            dist_last = ('dist_last','min')).reset_index()
v2DE2.head()

In [ ]:
#v2_DE2['prev_last'] = v2_DE2.sort_values(by=['month']).groupby(['year'])['dist_last','min'].shift(1)

# Write out each df as a csv file

In [ ]:
v2.to_csv('vaccines_delivered.csv', index = False)
v2DE.to_csv('DE vaccines delivered.csv', index = False)
v2_agg.to_csv('v2 aggregated.csv', index = False)